In [30]:
from typing import Dict, Tuple, List
from math import sqrt
from dataclasses import dataclass
from gurobipy import Model, GRB, tupledict
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
import numpy as np
import os
import re

In [31]:
class TspDataLoader:
    """ Class to load and process data for TSP instances. """

    def __init__(self, dir='data'):
        self.dir = dir

    def load_problem(self, problem):
        """ Load a TSP problem and return its vertices count and cost matrix. """
        file_name = os.path.join(self.dir, problem)

        # Read file content
        with open(file_name, 'r') as file:
            content = file.read()

        # Extract numeric part from the problem name
        numeric_part = re.search(r'\d+', problem).group()
        n = int(numeric_part)

        if n == 64:
            n = 65

        # Process the file content
        data_lines = content.split('\n')
        start_index = data_lines.index('EDGE_WEIGHT_SECTION')
        edge_weight_lines = data_lines[start_index + 1:][:-2]

        distance_matrix = [list(map(int, line.split())) for line in edge_weight_lines if line.strip()]
        # Flatten the entire distance matrix
        distance_list = [item for sublist in distance_matrix for item in sublist]

        my_matrix = np.array(distance_list).reshape((n, n))
        V = range(n)

        # Construct the cost dictionary
        cost = {(i, j): my_matrix[i, j] for i in V for j in V if i != j}

        return n, cost

In [32]:
class TspInstance:
    """ An instance of the Travelling Salesman Problem. """

    def __init__(self, n_vertices, cost, x=None, y=None):
        """
        Initialize a TSP instance.

        Parameters
        ----------
        n_vertices : int
            Number of vertices in the graph.
        cost : dict[tuple[int, int], float]
            Dictionary with arcs as key, and their corresponding costs as values.
        x : list[float], optional
            X coordinates of the vertices (for plotting).
        y : list[float], optional
            Y coordinates of the vertices (for plotting).
        """
        self.n_vertices = n_vertices
        self.cost = cost
        self.x = x if x is not None else []
        self.y = y if y is not None else []
        
@dataclass
class TspSolution:
    """ Solution to an instance of the Travelling Salesman Problem.

        Attributes
        ----------
        instance: TspInstance
            Reference to the TSP instance being solved.
        tour: List[int]
            Ordered list of vertices visited (without repeating the first one).
        cost: float
            Travel cost of the tour.
    """

    instance: TspInstance
    tour: List[int]
    cost: float

    def plot(self) -> Tuple[Figure, plt.Axes]:
        """ Plots the solution of the TSP.
        
            Vertices are black dots, the solution is represented by red lines.

            Returns
            -------
            A tuple with the figure and axes object from matplotlib.    
        """

        xy = [(self.instance.x[i], self.instance.y[i]) for i in self.tour + [self.tour[0]]]

        fig, ax = plt.subplots(figsize=(10, 10))
        ax.scatter(self.instance.x, self.instance.y, color='black')
        
        for pt1, pt2 in zip(xy[1:], xy[:-1]):
            x1, y1 = pt1
            x2, y2 = pt2
            ax.plot([x1, x2], [y1, y2], color='red')

        ax.set_title(f"TSP Solution. Num vertices: {self.instance.n_vertices}. Tour cost: {self.cost:.2f}.")
        ax.set_axis_off()

        return fig, ax
    
class TspIntegerBCSolver:
    """ 
    Solver for the Travelling Salesman Problem using the branch-and-cut algorithm.

    This solver applies the Dantzig-Fulkerson-Johnson (DFJ) formulation of the TSP
    and focuses on separating subtour elimination constraints (SECs) only on integer solutions.

    Attributes:
    _instance : TspInstance
        The instance of the TSP to be solved.
    _V : List[int]
        List of vertex indices in the TSP instance.
    _A : List[Tuple[int, int]]
        List of arcs (edges between pairs of vertices) in the TSP instance.
    _model : Model
        The Gurobi model for the TSP instance.
    _x : tupledict
        Gurobi variables representing whether an arc is included in the tour.
    """

    def __init__(self, instance: TspInstance, cardinality=None):
        """
        Initializes the solver with a TSP instance and builds the model.

        Parameters:
        instance (TspInstance): The TSP instance to be solved.
        """
        # Store the TSP instance and create lists of vertices and arcs
        self._instance = instance
        self._V = list(range(self._instance.n_vertices))
        self._A = list(self._instance.cost.keys())
        self.lazy_constraints_added = []  # Initialize an empty list to store lazy constraints
        self.cardinality = cardinality  # Store the cardinality parameter

        # Build the optimization model
        self.__build_model()

    def __build_model(self):
        """ 
        Builds the initial DFJ model for the TSP without subtour elimination constraints (SECs).
        """
        # Initialize the Gurobi model
        self._model = Model()

        # Add variables representing whether each arc is included in the tour
        self._x = self._model.addVars(self._A, obj=self._instance.cost, vtype=GRB.BINARY, name='x')

        # Add constraints to ensure exactly one outgoing and one incoming arc for each vertex
        self._model.addConstrs((self._x.sum(i, '*') == 1 for i in self._V), name='outgoing')
        self._model.addConstrs((self._x.sum('*', i) == 1 for i in self._V), name='incoming')

        # Conditionally add cardinality constraints if the cardinality is set to 2
        if self.cardinality == 2 or self.cardinality == 3:
            self._model.addConstrs((self._x[i, j] + self._x[j, i] <= 1 for i in self._V for j in self._V if i != j), name='cardinality two')

        # Add constraints to ensure x(i,j) + x(j,k) +x(k,i) <= 2
        if self.cardinality == 3:
            self._model.addConstrs((self._x[i, j] + self._x[j, k] + self._x[k, i]<= 2 for i in self._V for j in self._V for k in self._V if i != j and j != k and k != i), name='cardinality three')


    def __next_vertex(self, i: int) -> int:
        """
        Finds the next vertex in the current solution, starting from vertex i.

        Parameters:
        i (int): The starting vertex index.

        Returns:
        int: The index of the next vertex in the tour.
        """
        # Ensure the vertex index is valid
        assert 0 <= i < self._instance.n_vertices

        # Loop through all vertices to find the next one in the tour
        for j in self._V:
            if i != j:
                try:
                    # Inside a callback, retrieve the solution using cbGetSolution
                    val = self._model.cbGetSolution(self._x[i, j])
                except:
                    # Otherwise, access the variable's value directly
                    val = self._x[i, j].X

                # If the arc is part of the solution, return the next vertex
                if val > 0.5:
                    return j

        # If no successor is found, raise an error
        assert False, f"Vertex {j} has no successor?!"

    def __tour_staring_at(self, i: int) -> List[int]:
        """
        Constructs a tour (or subtour) starting at vertex i based on the current solution.

        Parameters:
        i (int): The starting vertex index.

        Returns:
        List[int]: A list of vertex indices representing the tour.
        """
        # Ensure the vertex index is valid
        assert 0 <= i < self._instance.n_vertices

        # Initialize the tour with the starting vertex
        tour = [i]
        current = self.__next_vertex(i)

        # Continue adding vertices to the tour until it loops back to the start
        while current != i:
            tour.append(current)
            current = self.__next_vertex(current)

        return tour

    def __add_sec_for(self, S: List[int]) -> None:
        """
        Adds a subtour elimination constraint (SEC) for the vertices in set S.

        Parameters:
        S (List[int]): A list of vertex indices forming a subtour.
        """
        # Ensure the subtour has a valid number of vertices
        assert 0 < len(S) < self._instance.n_vertices
   
        # Record the constraint information
        constraint_info = {
            'subtour': S,
            'constraint': sum(self._x[i, j] for (i, j) in self._A if i in S and j not in S) >= 1
        }

        self.lazy_constraints_added.append(constraint_info)
        
        # Add the actual lazy constraint to the model
        self._model.cbLazy(sum(self._x[i, j] for (i, j) in self._A if i in S and j not in S) >= 1)

    def __separate(self, where: int) -> None:
        """ Separates eventual violated SECs. """

        if where != GRB.Callback.MIPSOL:
            return
        
        # Set of vertices not yet placed in a tour
        remaining = set(self._V)

        while len(remaining) > 0:
            # Get any vertex in set `remaining`
            current = next(iter(remaining))

            # Get the subtour which includes the vertex
            subtour = self.__tour_staring_at(current)

            # If the tour visits all vertices, nothing to do
            if len(subtour) == self._instance.n_vertices:
                return

            # Otherwise, it's a subtour => Add a SEC
            self.__add_sec_for(subtour)

            # Update set `remaining`
            remaining -= set(subtour)

    def solve(self) -> TspSolution:
        """ Solves the TSP DFJ model and returns the solution.
        
            It throws a RuntimeError if Gurobi cannot solve the model to optimality.

            Returns
            -------
            A TspSolution object with details about the solution.
        """

        # We must set this parameter to use callbacks
        self._model.setParam(GRB.Param.LazyConstraints, 1)

        # Gurobi always passes two parameters to the callback: `model` and `where`
        self._model.optimize(lambda _, where: self.__separate(where))

        if self._model.Status != GRB.OPTIMAL:
            raise RuntimeError(f"Could not find the optimal solution. Gurobi status = {self._model.Status}")

        return TspSolution(
            instance=self._instance,
            tour=self.__tour_staring_at(0),
            cost=self._model.ObjVal
        )


## Baseline

In [34]:
# Load data
tsp_loader = TspDataLoader()
n_vertices, cost = tsp_loader.load_problem('rbg323.atsp')

tsp_loader_2 = TspDataLoader()
n_vertices_2, cost_2 = tsp_loader_2.load_problem('ftv64.atsp')

In [36]:
i = TspInstance(n_vertices=n_vertices_2, cost=cost_2)
s = TspIntegerBCSolver(instance=i, cardinality=None)
solution = s.solve()

Set parameter LazyConstraints to value 1
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 130 rows, 4160 columns and 8320 nonzeros
Model fingerprint: 0x1d8967fb
Variable types: 0 continuous, 4160 integer (4160 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+00, 3e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.01s
Presolved: 130 rows, 4160 columns, 8320 nonzeros
Variable types: 0 continuous, 4160 integer (4160 binary)

Root relaxation: objective 1.721000e+03, 119 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1721.00000    0    -          - 1721.00000      -     -    0s
     0     0 1771.00000    0    -          - 1

In [23]:
# Check how many times cardinality constraints where violated
for entry in s.lazy_constraints_added:
    subtour = entry['subtour']
    constraint = entry['constraint']
    
    # Check if the subtour length is exactly 2
    if len(subtour) == 2:
        # The subtour contains exactly two vertices i and j
        i, j = subtour
        # The corresponding constraint is for i to j and j to i
        print(f"Lazy constraint for subtour {subtour}")


Lazy constraint for subtour [35, 264]
Lazy constraint for subtour [0, 322]
Lazy constraint for subtour [1, 321]
Lazy constraint for subtour [2, 320]
Lazy constraint for subtour [3, 319]
Lazy constraint for subtour [4, 318]
Lazy constraint for subtour [5, 317]
Lazy constraint for subtour [6, 316]
Lazy constraint for subtour [7, 315]
Lazy constraint for subtour [8, 314]
Lazy constraint for subtour [9, 313]
Lazy constraint for subtour [10, 312]
Lazy constraint for subtour [11, 311]
Lazy constraint for subtour [12, 310]
Lazy constraint for subtour [13, 309]
Lazy constraint for subtour [14, 308]
Lazy constraint for subtour [15, 307]
Lazy constraint for subtour [16, 306]
Lazy constraint for subtour [17, 305]
Lazy constraint for subtour [18, 304]
Lazy constraint for subtour [19, 303]
Lazy constraint for subtour [20, 302]
Lazy constraint for subtour [21, 301]
Lazy constraint for subtour [22, 300]
Lazy constraint for subtour [23, 299]
Lazy constraint for subtour [24, 298]
Lazy constraint for su

In [24]:
# Check how many times cardinality constraints where violated
for entry in s.lazy_constraints_added:
    subtour = entry['subtour']
    constraint = entry['constraint']
    

    # Check if the subtour length is exactly 2
    if len(subtour) == 3:
        # The subtour contains exactly two vertices i and j
        i, j, k = subtour
        # The corresponding constraint is for i to j and j to i
        print(f"Lazy constraint for subtour {subtour}")

Lazy constraint for subtour [45, 88, 110]
Lazy constraint for subtour [162, 195, 275]
Lazy constraint for subtour [160, 161, 162]
Lazy constraint for subtour [0, 317, 316]
Lazy constraint for subtour [1, 308, 306]


## Cardinality 2

In [28]:
i = TspInstance(n_vertices=n_vertices, cost=cost)
s = TspIntegerBCSolver(instance=i, cardinality=2)
solution = s.solve()

Set parameter LazyConstraints to value 1
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 104652 rows, 104006 columns and 416024 nonzeros
Model fingerprint: 0x9be21f22
Variable types: 0 continuous, 104006 integer (104006 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 52003 rows and 0 columns
Presolve time: 0.35s
Presolved: 52649 rows, 104006 columns, 312018 nonzeros
Variable types: 0 continuous, 104006 integer (104006 binary)
Root relaxation presolved: 52730 rows, 104006 columns, 312984 nonzeros

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Concurrent spin time: 0.02s

Solved with primal simplex

Root relaxation: objective 1.326000e+03, 4245 iterations, 0.17 seconds (0.34 wor

## Cardinality 3

In [29]:
i = TspInstance(n_vertices=n_vertices, cost=cost)
s = TspIntegerBCSolver(instance=i, cardinality=2)
solution = s.solve()

Set parameter LazyConstraints to value 1
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 104652 rows, 104006 columns and 416024 nonzeros
Model fingerprint: 0x9be21f22
Variable types: 0 continuous, 104006 integer (104006 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 52003 rows and 0 columns
Presolve time: 0.34s
Presolved: 52649 rows, 104006 columns, 312018 nonzeros
Variable types: 0 continuous, 104006 integer (104006 binary)
Root relaxation presolved: 52730 rows, 104006 columns, 312984 nonzeros

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Concurrent spin time: 0.01s

Solved with primal simplex

Root relaxation: objective 1.326000e+03, 4245 iterations, 0.16 seconds (0.34 wor